In [1]:
import cv2
import numpy as np
import pyniryo
from pyniryo2 import Vision, NiryoRos

# Define the positions for the Tic Tac Toe field and turn indicators
positions = {
    "p1": (217, 154),
    "p2": (288, 148),
    "p3": (358, 141),
    "p4": (219, 226),
    "p5": (292, 215),
    "p6": (365, 210),
    "p7": (222, 300),
    "p8": (300, 291),
    "p9": (375, 285),
    "I1": (143, 182),  # Indication point for player 1
    "I2": (144, 292)   # Indication point for player 2
}

# Corrected HSV color ranges for red, green, and blue
color_ranges = {
    'red': ([0, 100, 100], [10, 255, 255]),    # Lower half of red hue (0-10)
    'green': ([35, 50, 50], [85, 255, 255]),    # Green (35-85)
    'blue': ([100, 150, 0], [140, 255, 255])    # Blue (100-140)
}

# Colors for drawing rectangles and contours
draw_colors = {
    'red': (0, 0, 255),
    'green': (0, 255, 0),
    'blue': (255, 0, 0)
}

# Initialize the Niryo robot's ROS instance and vision system
ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

# Function to detect color in a given position using surrounding pixels
def detect_color(hsv_frame, pos):
    x, y = pos

    # Define offsets for the surrounding pixels (1 center and 8 surrounding)
    offsets = [
        (0, 0),     # Center pixel
        (-1, 0),    # Left
        (1, 0),     # Right
        (0, -1),    # Top
        (0, 1),     # Bottom
        (-1, -1),   # Top-left
        (1, -1),    # Top-right
        (-1, 1),    # Bottom-left
        (1, 1)      # Bottom-right
    ]

    # Dictionary to count color occurrences
    color_counts = {color: 0 for color in color_ranges.keys()}
    color_counts['none'] = 0  # Initialize 'none' count

    # Check colors of the surrounding pixels
    for dx, dy in offsets:
        x_offset = x + dx
        y_offset = y + dy
        
        # Ensure we don't go out of bounds
        if 0 <= x_offset < hsv_frame.shape[1] and 0 <= y_offset < hsv_frame.shape[0]:
            pixel_color = hsv_frame[y_offset, x_offset]

            # Check against predefined color ranges
            for color_name, (lower, upper) in color_ranges.items():
                lower_bound = np.array(lower, dtype=np.uint8)
                upper_bound = np.array(upper, dtype=np.uint8)

                # Check if the pixel color is within the color range
                if all(lower_bound <= pixel_color) and all(pixel_color <= upper_bound):
                    color_counts[color_name] += 1
                    break  # Stop checking once we find a match
            else:
                color_counts['none'] += 1  # If no color matched

    # Determine the most frequent color detected
    detected_color = max(color_counts, key=color_counts.get)
    return detected_color

# Function to overlay a color mask for detected chips
def overlay_color_mask(frame, pos, color_name, region_size=10):
    x, y = pos
    # Define the region of interest for drawing the rectangle
    x_start = max(0, x - region_size)
    y_start = max(0, y - region_size)
    x_end = min(frame.shape[1], x + region_size)
    y_end = min(frame.shape[0], y + region_size)

    # Get the drawing color (RGB) based on detected color name
    color = draw_colors.get(color_name, (255, 255, 255))  # Default to white if color is 'none'

    # Draw a filled rectangle over the position to indicate the detected color
    cv2.rectangle(frame, (x_start, y_start), (x_end, y_end), color, thickness=-1)

# Function to find and draw contours around detected color regions
def draw_contours_around_color(frame, hsv_frame, color_name):
    if color_name not in color_ranges:
        return
    
    # Extract color range for the specified color
    lower_bound = np.array(color_ranges[color_name][0], dtype=np.uint8)
    upper_bound = np.array(color_ranges[color_name][1], dtype=np.uint8)

    # Create a mask for the detected color
    mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours around the detected objects
    for contour in contours:
        if cv2.contourArea(contour) > 100:  # Only consider contours with significant area
            cv2.drawContours(frame, [contour], -1, draw_colors[color_name], 2)

# Initialize a state dictionary to store current colors of each position
state = {pos_name: "none" for pos_name in positions.keys()}

while True:
    # Capture the image from the Niryo robot camera
    img_compressed = vision.get_img_compressed()
    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    # Resize the frame for display and processing
    frame = cv2.resize(img, (640, 480))

    # Convert the image to HSV for better color detection
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Loop through each position (Tic Tac Toe and indication points) and check for color
    for pos_name, pos in positions.items():
        detected_color = detect_color(hsv_frame, pos)

        if detected_color != state[pos_name]:  # Check if the color has changed
            print(f"State change at {pos_name}: {state[pos_name]} -> {detected_color}")  # Log the state change
            state[pos_name] = detected_color  # Update the state

        if detected_color != "none":
            # Overlay a color mask on the position with the detected color
            overlay_color_mask(frame, pos, detected_color)
            
            # Drawing contour around detected objects of that color
            draw_contours_around_color(frame, hsv_frame, detected_color)

        # Display the detected color as text at the position
        cv2.putText(frame, f"{pos_name}: {detected_color}", pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display the frame with color masks, contours, and detected colors
    cv2.imshow('Tic Tac Toe Color Detection', frame)

    # Check if 'q' key has been pressed to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup resources
cv2.destroyAllWindows()


State change at p9: none -> red
State change at p9: red -> none
State change at p9: none -> red
State change at p9: red -> none
State change at p6: none -> red
State change at p6: red -> none
State change at p3: none -> red
State change at p3: red -> none
